In [1]:
from pybacktestchain import data_module
from pybacktestchain import broker
from pybacktestchain.data_module import * 
from datetime import date, timedelta, datetime


from my_package_project.data_treatment import *
from my_package_project.graphs  import *
from my_package_project.operations import *




In [2]:
import kaleido

In [3]:
tickers = ['SPY', 'TLT', 'GLD']  # S&P 500 Index, 20+ Year Treasury Bond, Gold (commodities)
data = get_stocks_data(tickers=tickers, start_date='2017-02-28', end_date='2020-12-31').dropna()
# data['Date'] = data['Date'].astype(str).str[:10] #we had an error with non-UTC dates
# data['Date'] = pd.to_datetime(data['Date'])
data

,Date,Open,High,Low,Close,Adj Close,Volume,ticker
0,2017-02-28 00:00:00-05:00,236.669998,236.949997,236.020004,236.470001,207.125259,96961900,SPY
1,2017-03-01 00:00:00-05:00,238.389999,240.320007,238.369995,239.779999,210.024521,149158200,SPY
2,2017-03-02 00:00:00-05:00,239.559998,239.570007,238.210007,238.270004,208.701904,70246000,SPY
3,2017-03-03 00:00:00-05:00,238.169998,238.610001,237.729996,238.419998,208.833282,81974300,SPY
4,2017-03-06 00:00:00-05:00,237.500000,238.119995,237.009995,237.710007,208.211411,55391500,SPY
...,...,...,...,...,...,...,...,...
963,2020-12-23 00:00:00-05:00,175.100006,176.210007,175.059998,175.649994,175.649994,6542800,GLD
964,2020-12-24 00:00:00-05:00,175.550003,176.369995,175.509995,176.350006,176.350006,3695400,GLD
965,2020-12-28 00:00:00-05:00,177.259995,177.910004,175.630005,175.710007,175.710007,7778700,GLD
966,2020-12-29 00:00:00-05:00,176.250000,176.970001,175.570007,176.350006,176.350006,5983700,GLD


In [4]:
data_module = DataModule(data=data)
info = Information(s = timedelta(data_module.data.size/len(tickers)), data_module=data_module)
t = datetime(2020,12,29)
moments = FirstTwoMoments.compute_information(info, t=t)


In [5]:
riskparityportfolio =  data_treatment.RiskParity.compute_portfolio_riskparity(info, t=t, information_set=moments)
riskparityportfolio

{'GLD': 0.3334886071418446,
 'SPY': 0.23892092775729304,
 'TLT': 0.4275904651008624}

In [6]:
vol = data_treatment.portfolio_volatility(riskparityportfolio,information_set=moments)
vol

20.401651144872268

In [7]:
data_treatment.RiskParity.compute_portfolio_riskparity_voltarget_leverage(info, t=t, information_set=moments, target_volatility=vol, leverage_factor=1)

INFO:root:Original Volatility: 20.401651144872268


{'GLD': 0.3334886071418446,
 'SPY': 0.23892092775729304,
 'TLT': 0.4275904651008624}

In [8]:
visualizer = PortfolioVisualizer(portfolio=riskparityportfolio, information_set=moments)
visualizer.plot_portfolio_weights()


In [9]:
data_treatment.compute_risk_contributions(riskparityportfolio, moments)

{'GLD': 0.333, 'SPY': 0.333, 'TLT': 0.333}

In [10]:
visualizer.plot_risk_allocation_pie()

In [11]:
test = Backtest_up(
    initial_date = datetime(2017, 1, 1),
    final_date = datetime(2018, 12, 31),
    information_class = FirstTwoMoments,
    risk_model=StopLoss,
)

In [12]:
test.run_backtest()

INFO:root:Running backtest from 2017-01-01 00:00:00 to 2018-12-31 00:00:00.
INFO:root:Retrieving price data for universe
/Users/marc/Desktop/anaconda3/envs/pybacktestchain/lib/python3.11/site-packages/pandas/core/frame.py:11211: RuntimeWarning:

Degrees of freedom <= 0 for slice

/Users/marc/Desktop/anaconda3/envs/pybacktestchain/lib/python3.11/site-packages/numpy/lib/function_base.py:2748: RuntimeWarning:

divide by zero encountered in divide

/Users/marc/Desktop/anaconda3/envs/pybacktestchain/lib/python3.11/site-packages/numpy/lib/function_base.py:2748: RuntimeWarning:

invalid value encountered in multiply

INFO:root:-----------------------------------
INFO:root:Rebalancing portfolio at 2017-01-31 00:00:00
/Users/marc/Desktop/Master 203/M2/S1/Python/Project/pybacktestchain/src/pybacktestchain/broker.py:115: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when 

Risk Contributions: {'GLD': 0.333, 'SPY': 0.333, 'TLT': 0.333}


Portfolio Volatility: 9.96%
Annualized Return: 2.26%


INFO:root: Backtest completed. Final portfolio value: 1098226.6372833252
